***Задание*** А: к этапу предобработки текстов добавить фильтр по частям речи. То есть после предобратки в текстах должны оставаться только слова определенных частей речи. Попробовать: 1) только сущ., 2) сущ.+прил., 3) сущ.+прил.+гл. 4) свой вариант. Этот эта предобработки соединить со спобами представления текста через вектор:

1) мешок слов,2) tf-idf, 3) LSI 4) LDA
2) Использовать 2 алгорится классификации, которые лучше всегосебя показали в предыдущих исследованиях.

In [1]:
import warnings
import numpy as np
import re
import string
import nltk
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from tqdm.notebook import tqdm
from gensim import corpora, models
from nltk import pos_tag
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from catboost import CatBoostClassifier, Pool
warnings.filterwarnings("ignore")

/Users/igorkopylov/PycharmProjects/liner_model/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Формирование текстов

In [2]:
from sklearn.datasets import fetch_20newsgroups
number_of_topic = 3
newsgroups_train = fetch_20newsgroups(subset='train')
indices = np.where((newsgroups_train.target >= 0) & (newsgroups_train.target <= number_of_topic))[0]
texts = [newsgroups_train.data[i] for i in indices]
target = np.array([newsgroups_train.target[i] for i in indices])
print(f"Всего элементов: {target.shape[0]}")
for i in range(number_of_topic):
    count = np.count_nonzero(target == i)
    print(f"ко-во элементов из класса {i}: {count}")

Всего элементов: 2245
ко-во элементов из класса 0: 480
ко-во элементов из класса 1: 584
ко-во элементов из класса 2: 591


In [3]:
def spacy_lemmatize_tokens(tokens):
    
    nlp = spacy.load("en_core_web_sm")

    # Лемматизация слова "running"
    doc = nlp(' '.join(tokens))
    lemmas = [token.lemma_ for token in doc]
    return lemmas

def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmatized_tokens


def preprocess_text(text): 
    # Удаление пунктуации
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatized_tokens = lemmatize_tokens(tokens)  #spacy_lemmatize_tokens(tokens)
    return lemmatized_tokens

In [4]:
tokenized_corpus = [preprocess_text(i) for i in texts]
tokenized_corpus[0][:5]

['From', 'jgreenamber', 'Joe', 'Green', 'Subject']

In [5]:
nltk.download('averaged_perceptron_tagger')

def get_part_speech(token_list, entities):
    # 1) только сущ., 2) сущ.+прил., 3) сущ.+прил.+гл. 4) 
    # N - noun; V - verb; JJ - adjectives
    pos_tags = pos_tag(token_list, lang='eng')
    part_speech = []
    for word, pos in pos_tags:
        if any(pos.startswith(j) for j in entities):
           part_speech.append(word)
    return part_speech

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 60] Operation timed out>


In [6]:
tokenized_corpus_n = [get_part_speech(i, ['N']) for i in tokenized_corpus]
tokenized_corpus_na = [get_part_speech(i, ['N', 'JJ']) for i in tokenized_corpus]
tokenized_corpus_nav = [get_part_speech(i, ['N', 'JJ', 'V']) for i in tokenized_corpus]
tokenized_corpus_nin = [get_part_speech(i, ['N', 'IN']) for i in tokenized_corpus]
# 'IN' - предлоги

In [7]:
tokenized_corpuses = [tokenized_corpus_n, tokenized_corpus_na, tokenized_corpus_nav, tokenized_corpus_nin]

In [8]:
tokenized_corpus_n[0][:10]

['jgreenamber',
 'Joe',
 'Green',
 'Subject',
 'Re',
 'Weitek',
 'P9000',
 'Organization',
 'Harris',
 'Computer']

In [9]:
tf_idf_corpuses = []
for corp in tokenized_corpuses:
    documents = []
    for doc in corp:
        documents.append(' '.join(doc))
    tfidf_vectorizer = TfidfVectorizer()
    tf_idf_corpuses.append(tfidf_vectorizer.fit_transform(documents))

In [10]:
count_words_corpuses = []
for corp in tokenized_corpuses:
    documents = []
    for doc in corp:
        documents.append(' '.join(doc))
    vectorizer = CountVectorizer()
    count_words_corpuses.append(vectorizer.fit_transform(documents).toarray())

In [11]:
freq_words_corpuses = []
for corp in count_words_corpuses:
    documents = []
    for doc in corp:
        total_words_per_document = np.sum(doc)
        word_frequency_matrix = doc / total_words_per_document
        documents.append(word_frequency_matrix)
    freq_words_corpuses.append(documents)

# Cкармливание документов моделям

lda:

In [12]:
alpha_values = ['symmetric', 'asymmetric', 0.1]
eta_values = ['auto', 0.1, 0.01]

# alpha контролирует разнообразие тем в документах
# eta влияет на разнообразие слов в темах

models = {}
dictionary = Dictionary(tokenized_corpus_nav)

bow_corpus = [dictionary.doc2bow(text) for text in tokenized_corpus_nav]

for alpha in alpha_values:
    for eta in eta_values:
        model_key = f"alpha={alpha}, eta={eta}"  # tokenized_corpus_nav
        models[model_key] = LdaModel(
            corpus=bow_corpus,
            id2word=dictionary,
            num_topics=3, 
            random_state=100,
            update_every=1,
            chunksize=100,
            passes=10,
            alpha=alpha,
            eta=eta
        )

for model_key, model in models.items():
    print(f"\nМодель с {model_key}:")
    for idx, topic in model.print_topics(-1):
        print(f"Топик: {idx} \nСлова: {topic}")



Модель с alpha=symmetric, eta=auto:
Топик: 0 
Слова: 0.006*"writes" + 0.006*"Re" + 0.006*"God" + 0.005*"Subject" + 0.005*"people" + 0.005*"Lines" + 0.005*"Organization" + 0.004*"article" + 0.004*"dont" + 0.004*"think"
Топик: 1 
Слова: 0.010*"Subject" + 0.010*"Lines" + 0.009*"Organization" + 0.008*"drive" + 0.005*"file" + 0.005*"University" + 0.005*"card" + 0.004*"Windows" + 0.004*"use" + 0.004*"Re"
Топик: 2 
Слова: 0.063*"MAXAXAXAXAXAXAXAXAXAXAXAXAXAXAX" + 0.011*"M" + 0.009*"P" + 0.002*"D" + 0.001*"px" + 0.001*"MG9VG9VG9VG9VG9VG9VG9VG9VG9VG9VG9VG9VG9VG9VG9V" + 0.001*"MP" + 0.001*"H" + 0.001*"MO" + 0.001*"G"

Модель с alpha=symmetric, eta=0.1:
Топик: 0 
Слова: 0.016*"drive" + 0.007*"Subject" + 0.007*"writes" + 0.006*"article" + 0.006*"Lines" + 0.006*"Re" + 0.006*"system" + 0.006*"Organization" + 0.005*"dont" + 0.005*"know"
Топик: 1 
Слова: 0.014*"Subject" + 0.014*"Lines" + 0.014*"Organization" + 0.009*"file" + 0.008*"University" + 0.007*"Windows" + 0.006*"card" + 0.006*"driver" + 0.006

In [13]:
def lda_repo(tokenized_corpus, num_topics=15):
    # Создание словаря
    dictionary = Dictionary(tokenized_corpus)

    bow_corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

    lda_model = LdaModel(
        corpus=bow_corpus,  
        id2word=dictionary,  
        num_topics=num_topics,  # Количество тем
        passes=10,  # Количество итераций по корпусу
        iterations=100,  # Количество итераций для оценки параметров
        random_state=42  # Зафиксированный случайный seed для воспроизводимости результатов
    )
    return lda_model, bow_corpus

In [14]:
best_param = 10 # 1810
lda_vectors = []
for tokenized_corpus in tokenized_corpuses:
    lda_model, bow_corpus = lda_repo(tokenized_corpus, best_param)
    # получаем тематические векторы
    lda_vec = []
    for i in tqdm(range(len(target))):
        doc_topics = lda_model.get_document_topics(bow_corpus[i])
        doc_vec = np.zeros(best_param)
        for topic, prob in doc_topics:
            #top_numb = int(re.sub(r'[a-zA-Z]', '', topic))
            doc_vec[topic] = prob
        lda_vec.append(doc_vec)
    lda_vectors.append(lda_vec)

  0%|          | 0/2245 [00:00<?, ?it/s]

  0%|          | 0/2245 [00:00<?, ?it/s]

  0%|          | 0/2245 [00:00<?, ?it/s]

  0%|          | 0/2245 [00:00<?, ?it/s]

lsi:

In [17]:
mini_size = 1646  # 1646
lsi_represent = []
for tokenized_corpus in tf_idf_corpuses:
    lsi_model = TruncatedSVD(n_components=mini_size)  # Выбор количества компонентов
    lsi_vectors = lsi_model.fit_transform(tokenized_corpus)
    lsi_represent.append(lsi_vectors)

# Обучение моделей

## GBM

In [18]:
def learning_gbm(data, target, iter=150, plot_val=False):
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
    train_pool = Pool(X_train, label=y_train)
    test_pool = Pool(X_test, label=y_test)
    
    CatBoost = CatBoostClassifier(iterations=iter, learning_rate=0.1, random_seed=42, loss_function='MultiClass')
    CatBoost.fit(train_pool, eval_set=test_pool, verbose=False, plot=plot_val)
    y_pred = CatBoost.predict(X_test)
    f2_score = fbeta_score(y_test, y_pred, beta=2, average='macro')
#    print("F2-мера:", f2_score)
    return f2_score 

### LSI

In [19]:
learning_gbm(lsi_represent[0], target, iter=150, plot_val=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0.8499536997611281

In [20]:
learning_gbm(lsi_represent[0], target, iter=50)  # noun

0.8252944313847022

tokenized_corpuses = [tokenized_corpus_n, tokenized_corpus_na, tokenized_corpus_nav, tokenized_corpus_nin]

In [21]:
learning_gbm(lsi_represent[1], target, iter=50)  # noun adject

0.8189387186793669

In [22]:
learning_gbm(lsi_represent[2], target, iter=50)  # noun adj verb

0.8395561051791793

In [23]:
learning_gbm(lsi_represent[3], target, iter=50)  # noun proposition

0.8123624930928786

### Lda 

In [24]:
learning_gbm(lda_vectors[0], target, iter=150, plot_val=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0.6479268669281357

In [25]:
results = [learning_gbm(lda_vectors[i], target, iter=50) for i in range(4)]
results

[0.6431253372332889,
 0.6085763079998462,
 0.6615513917228233,
 0.5313534262164537]

### A bag of words

In [26]:
learning_gbm(count_words_corpuses[0], target, iter=150, plot_val=True)  # noun proposition

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0.8386888912581807

In [27]:
results = [learning_gbm(count_words_corpuses[i], target, iter=50) for i in range(4)]  # noun proposition

In [28]:
results

[0.7535797600100154,
 0.7752726268841157,
 0.7625387150061051,
 0.7697270000631541]

### freq_words_corpuses

In [29]:
#freq_words_corpuses

In [30]:
learning_gbm(freq_words_corpuses[0], target, iter=150, plot_val=True)  # noun proposition

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0.8340160774896315

In [31]:
results = [learning_gbm(freq_words_corpuses[i], target, iter=50) for i in range(4)]  # noun proposition

In [32]:
results

[0.7486128834086969,
 0.7718663462184916,
 0.7744106709841607,
 0.7687470379309826]

In [ ]:
# 